## PageRank 实验 _ solutions

### Problem 1

In [1]:
import numpy as np

def to_matrix(filename, n):
    """Return the n x n adjacency matrix described by datafile.

    Parameters:
        datafile (str): The name of a .txt file describing a directed graph.
        Lines describing edges should have the form '<from node>\t<to node>\n'.
        The file may also include comments.
        n (int): The number of nodes in the graph described by datafile

    Returns:
        An adjacency matrix (ndarray).
    """
    edge_list = []    # 初始化一个列表, 用于存储边
    
    adj = np.zeros((n, n), dtype=int)    # 初始化邻接矩阵
   
    with open(filename, 'r') as myfile:
        for line in myfile:
            edge_list.append(line.strip().split())
    
    for edge in edge_list:
        
        # 如果存在顶点edge[0] -> e顶点edge[1]的边,
        # 则邻接矩阵的edge[0]行、edge[1]列的值为1
        
        # 异常处理: edge_list列表中可能包含文件中的文字说明部分        
        try:
            adj[int(edge[0]), int(edge[1])] = 1
        except:
            continue

    return adj
    
to_matrix('matrix.txt', 8)

array([[0, 0, 0, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0, 1, 1, 0],
       [1, 0, 0, 0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0]])

### Problem 2

In [2]:
def calculateK(A,N):
    """Compute the matrix K as described in the lab.

    Parameters:
        A (ndarray): adjacency matrix of a gragh
        N (int): the number of nodes in the graph

    Returns:
        K (ndarray)
    """
    
    K = np.zeros((N,N))    # 初始化转移矩阵
    
    # 按定义计算转移矩阵K, 没有采用实验文档描述的方法
    for i in range(N):     
        out = A[i].sum()    # 计算顶点的出度
        
        # 如果存在sink顶点, 修改邻接矩阵
        if (out == 0):
            A[i] = np.ones(N)
            out = N
        
        for j in range(N):     
            if (A[i, j] == 1):          
                K[j, i] = 1 / out
        
    return K

def unmodify_calculateK(A,N):
    """计算转移矩阵K, 不修改邻接矩阵, 即不处理sink顶点.

    Parameters:
        A (ndarray): adjacency matrix of a gragh
        N (int): the number of nodes in the graph

    Returns:
        K (ndarray)
    """
    
    K = np.zeros((N,N))    # 初始化转移矩阵
    
    # 计算转移矩阵, 按定义计算, 没有采用实验文档描述的方法
    for i in range(N):       
        out = A[i].sum()    # 计算顶点的出度
        
        for j in range(N):          
            if (A[i, j] == 1):     
                K[j, i] = 1 / out
        
    return K

A = to_matrix('matrix.txt', 8)
calculateK(A, 8)

array([[0.        , 1.        , 0.125     , 0.33333333, 0.33333333,
        0.5       , 1.        , 1.        ],
       [0.        , 0.        , 0.125     , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.125     , 0.33333333, 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.125     , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.125     , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.125     , 0.        , 0.33333333,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.125     , 0.33333333, 0.33333333,
        0.5       , 0.        , 0.        ],
       [1.        , 0.        , 0.125     , 0.        , 0.        ,
        0.        , 0.        , 0.        ]])

### Problem 3

In [3]:
def less_than_tol(err_list, tol): 
    """To check if all the value in |P(t+1)-P(t)| are
    less than a given tolerance 'tol'.
    
    Parameters:
        err_list (list): |P(t+1)-P(t)|.
        tol (float): the given tolerance value.
        
    Returns: Boolean.    
    """
       
    for x in err_list:

        if tol <= x: 
            
            return False
        
    return True

def iter_solve(adj, max_iter=100, d=0.85, tol=1e-5):
    """Return the page ranks of the network described by 'adj'.
    Iterate through the PageRank algorithm until the error is less than 'tol'.

    Parameters:
        adj (ndarray): The adjacency matrix of a directed graph.
        max_iter (int): the Maximum number of iterations.
        d (float): The damping factor, a float between 0 and 1.
        tol (float): Stop iterating when the change in approximations to
        the solution is less than 'tol'.

    Returns:
        The approximation to the steady state.
    """
    n = adj.shape[0]    # n is the number of nodes
    
    I = np.ones(n).reshape(-1, 1)    
    
    # 两种初始化方法，任选一种
    # p = 1 / n * I    # 均匀初始化 pagerank 值
    p = np.random.rand(n).reshape(-1,1)    # 随机初始化 pagerank 值
    
    # 计算转移矩阵K
    K = calculateK(adj, n)
    # K = unmodify_calculateK(adj, n)
    
    for i in range(max_iter):
        
        prev_p = p    
        
        # pagerank 公式, p --> p(t+1), prev_p --> p(t)
        p = d * np.matmul(K, prev_p) + (1 - d) / n * I
        
        err_list = [abs(p[j] - prev_p[j]) for j in range(n)]    # |P(t+1) - P(t)|
        
#         if sum(err_list) < n*tol:
#             return p
        # 检查err_list中的每一项是否比允许的tol值小
        if less_than_tol(err_list, tol):          
            return p
        
    return p
    
def main():

    print("\033[1m顶点0到顶点7的PageRank值(PR值):\n\033[0m")
    
    adj = to_matrix("matrix.txt", 8)
    
    print(iter_solve(adj))
    
if __name__ == '__main__':
    main()

顶点0到顶点7的PageRank值(PR值):

[[0.43870932]
 [0.02171029]
 [0.02786154]
 [0.02171029]
 [0.02171029]
 [0.02786154]
 [0.04585394]
 [0.39460966]]


<font color = blue>用unmodify_calculateK（第44行）替换calculateK（第43行），重新运行程序，对比两次输出的PR值，结合图结构从直观上进行分析。</font>  
<font color = blue>提示1：顶点0的PR值是最大的，顶点7的值第二大，顶点1、3、4的PR值相同且最小，顶点2、5的PR值相同。</font>  
<font color = blue>提示2：实验文档中NetworkX部分代码有误，请自行更正。</font>

### Problem 4

In [4]:
import numpy as np

def team_rank(filename='ncaa2013.csv'):
    """Use iter_solve() to predict the rankings of the teams in the given
    dataset of games. The dataset should have two columns, representing
    winning and losing teams. Each row represents a game, with the winner on
    the left, loser on the right. Parse this data to create the adjacency
    matrix, and feed this into the solver to predict the team ranks.

    Parameters:
        filename (str): The name of the data file.
    Returns:
        ranks (list): The ranks of the teams from best to worst.
        teams (list): The names of the teams, also from best to worst.
    """
    
    all_games = []
    with open('ncaa2013.csv', 'r') as ncaafile:
        ncaafile.readline()    # reads and ignores the header line
        for line in ncaafile:
            all_games.append(line.strip().split(','))    # split on commas
    
    # 统计共有多少支球队
    all_teams = set()
    for item in all_games:
        all_teams.update(item)
    num = len(all_teams)
    
    # 给每支球队分配一个int的ID号
    team_to_ID = {}
    ID = 0
    for item in all_teams:
        team_to_ID[item] = ID
        ID += 1
    
    # 初始化邻接矩阵
    adj = np.zeros((num, num), dtype=int)
    
    # 计算邻近矩阵
    for item in all_games:
        adj[[team_to_ID[item[1]]], team_to_ID[item[0]]] = 1
    
    # 计算PageRank值
    pagerank = iter_solve(adj, d=0.7)    
    
    # 把每支球队和它的pagerank值保存为字典
    team_to_pagerank = {}
    i = 0
    for k in team_to_ID.keys():
        team_to_pagerank[k] = pagerank[i][0]
        i += 1
        
    # 按pagerank值给球队排名
    teams_ranks = sorted(team_to_pagerank.items(), key=lambda item: item[1],
                        reverse=True)
    
    teams = [x[0] for x in teams_ranks] 
    ranks = [x[1] for x in teams_ranks]
    
    return ranks, teams

def main():

    print("\033[1mThe Top 5 teams and ranks are:\033[0m")
    
    ranks, teams = team_rank()
    
    for i in range(5):
        print(teams[i], "\t", ranks[i])

if __name__ == '__main__':
    main()

The Top 5 teams and ranks are:
Duke 	 0.009676031477973064
Butler 	 0.008549790147970566
Louisville 	 0.008508874610734454
Illinois 	 0.008347320815693218
Indiana 	 0.008238267781649154
